STA 663 Final Project: Scalable K-Means++ 
====

Shuaiqi Zhang & Tongrong Wang
----

In [1]:
import numpy as np
from pandas import Series, DataFrame
from multiprocessing import Pool, Value, Array
import multiprocessing as mp
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
%load_ext cython

1 Introduction
==

Since 1967，K-means has remained as one of the most important unsupervised learning methodology. The advantage of this algorithm is its simplicity: starting with a set of randomly chose inital centers, one repeatedly assigns each inputpoint to its nearesr center, and then recomputes the centers given the points assigned. However, k-means has its drawbacks which are efficiency and quality. More specifically, a poorly selected initialization of this method would compromise the method efficient. K-means++ was brought up to improve this scenario by choosing a good set of strating points. However, with the approaching of the are of big data, the recently proposed initialization algorithm, K-means++, become overly computational intensive because of its inherent sequential nature which icreases the difficulty of implement of parralling coding. An revised version of it called K-meansII are addressed to improve the efficient in both sequential and parallel settings for large-scaled data. The superiority is demonstrated by a simulation written below. 


2 Algorithm
==

2.1 Notation
--

Let X={ $x_1,...,x_n$ } be a set of points in the d-dimensional Euclidean space and let k be a positive integer specifying the number of clusters. Let $ || x_i \times x_j || $ denote the Euclidean distance between $x_i$ and $x_j$. For a point x and a subset Y $ \subseteq $ X of points, the distance is defined as d(x,Y) = $\min_{ y \in Y} || x \times y || $. For a subset Y $ \subseteq $ X of points, let its centroid be given by
$$
centroid(Y)=\frac{1}{|Y|}\sum_{y \in Y} y
$$

Let C={ $c_1,...,c_k $} be a set of points and let $Y \subseteq X$. We define the cost of Y with respect of C as,

$$
\phi_Y (C) = \sum_{y \in Y}d^2(y,C)=\sum_{y \in Y}\min_{i=1,...k} ||y-c_i||^2
$$

The goal of K-means algorithm is to find a set of centers C to minimize the cost $\phi_Y (C)$.

An old fashion way to find the optimized cost starting with a random set of k centers. In each iteration, a clustering of X is derived from the current set of centers. The centroids of these derived clusters then become the centers for the next iteration. The iteration is then repeated until a stable set of centers is obtained. The iterative portion of the above method is called Lloyd’s iteration.


2.2 K-Means++
--

The intuition behind the K-Means++ is to select an initial set of centers which scatter as wide as possisble. This is realized by a non-uniform distribution. This algorithm has k iterations and selects one point in each iteration based on their distance from the centers gained form the pervious interation. The longer distance leads to a higher probability of beening chosen. The deataied steps are listed as,
1. C $\gets$ sample a point uniformly at random from X
2. **While** |C| < k **do**:
3.  Sample $x\in X$ with probablility $\frac{d^2(x,C)}{\phi_X (C)}$
4. C $\gets$ C$\cup${x}
5. **end While**

The central drawback of k-means++ initialization from a scalability point of view is its inherent sequential nature: the choice of the next center depends on the current set of centers.

2.3 K-MeansII
--

The algorithm K-MeansII is an alternative parapproach for initializeing inspired by k-means++. It is easy to be parallelized and has same provable approximation guarantees as the former. First deine an oversampling factor $l=\Omega (k)$, a function of the center number. The detalied steps are listed as below,
1. C $\gets$ sample a point uniformly at random from X
2. $\psi \gets \phi_X (C)$
3. **for** $O(log(\psi))$ times **do**:
4. C'$\gets$ sample each point $x\in X$ independently with probablility $\frac{l \cdot d^2(x,C)}{\phi_X (C)}$ 
5. $C\gets C\cup C'$
6. **end for**
7. For $x\in C$, set $w_x$ to be the number of points in X closer to x than any other point in C
8. Recluster the weighted points in C into k clusters using K-Means++ algorithm

the expected number of points chosen in each iteration is $l$ and at the end, the expected number of points in C is $O(log(\psi))$ , which is typically more than k. To reduce the number of centers, Step 7 assigns weights to the points in C and Step 8 reclusters these weighted points to obtain k centers. We can easily tell from the logitic of the algorithm that the it could be easily parallelized from step3 to step6.

3 Code Structure and Simulated Data
====

3.1 Simulated Data
--

The simulated dataset used for testing the code was sampled from a d-dimensional spherical Gaussian distribution. First, k datapoints {$c_1,...,c_k$} were sampled with mean atthe origin and variance R $\in$ {1,10,100}. Second, for each $c_i$'s,  n datapoints were sampled from spherical Gaussian distribution with mean $c_i$ and unit vatiance.  For each datapoint, we labeled it with the index of the center, so that we could test the false-classified rate. Given the k centers, this is a mixture of k spherical Gaussians with equal weights. 

In this section, we test the code using simulated data in respect of running time and model cost for each initialization algorithm. We set the dimension of the spherical Gaussian distribution equals to 15 and center number equals to 20.

In [6]:
#simulation
def simudata(ncenter,nsize,dim):
    mean=[0]*dim
    cov=np.diag(np.random.choice([1,10,100],dim,replace=True))
    center=np.random.multivariate_normal(mean, cov, ncenter)
    s=np.random.multivariate_normal(center[0,], np.eye(dim),nsize)
    sample=np.concatenate((s, 0*np.ones(nsize)[:,None]), axis=1)
    for i in range(ncenter-1):
        s=np.random.multivariate_normal(center[i+1,], np.eye(dim),nsize)
        s=np.concatenate((s, (i+1)*np.ones(nsize)[:,None]), axis=1)
        sample=np.concatenate((sample,s),axis=0)
    return center, sample
center, sample=simudata(20,1000,15)
sample
data=sample[:,0:15]
DataFrame(sample).ix[0:5,:]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-20.279973,6.168723,15.039032,-1.523202,1.285585,10.082722,5.691313,-0.277854,-0.133877,3.966106,-1.981335,-1.139734,-20.253003,3.441818,0.427913,0
1,-19.503421,5.750836,13.744940,-0.711089,-0.338485,7.878700,5.567016,-1.278351,2.141314,5.482336,-1.540781,-1.388180,-19.639221,2.172995,-0.626806,0
2,-20.246589,5.993672,12.123255,-1.039364,2.117176,10.179287,4.732125,-0.178207,4.439590,5.618383,-3.150249,-0.587217,-22.121103,2.726640,-0.537216,0
3,-21.480357,5.171614,14.172628,-3.995795,-0.318042,9.330227,4.056415,-0.131168,0.730610,4.217022,-2.896615,-0.191425,-18.559160,1.770672,0.290932,0
4,-23.333672,5.877585,16.052378,-2.524240,2.010482,9.724667,4.193206,-0.271410,3.245799,4.386212,-2.354982,-0.697349,-19.791202,2.133626,-1.516168,0
5,-20.326092,4.576167,15.308598,-2.176692,-0.373238,8.479428,4.044674,-0.785489,1.584038,5.788703,-1.260462,-0.681839,-22.416479,1.804593,0.925291,0


3.2 Code prifilling and Optimization
--

- Python function
     * simudata(ncenter,nsize,dim):Simulates the data set, where ncenter is the number of centers, and nsize and dim give the dimension of the data. It returns a data set.
     - kmeans(data, ncenter, center, maxiter=1000, maxtol=10e-5):Provides the traditional kmeans algorithm, the input is the data, number of centers and the initial centers. The default maximum iteration  and convergent tolerance is 1000 and 10e-5. It returns the centers,clusters and iteration number.
     - kpp(x,k,weight=1): Provides the initial centers using K-Means++, where x is the data, k is the number of center, and the weight is set as 1. It returns the initial center.
     - kII(x,k,l): Provides the initial centers using K-MeansII, where x is the data, k is the number of center, and l is the oversampling factor, which is a function of k. It returns the initial center.
- Optimization
     * Cython version for all functions, and those cython function return the same thing as the python function
       * function names: simudata_cy,kmeans_cy,kpp_cy,kII_cy
     - Parellel computing for K-meanII algorithm 
       * function name: KII_pal
- Other functions
     * dis(a,b): returns the distant for any two matrix
     - cost(a,b):returns $\phi_a (b)$
  

The main function is KII. This function uses K-meansII to generate initial starting points. The input of the function are data x, number of centers k and oversampling factor l, which is a function of k. It will produce the initical center. KII function contains the dis function, which is used to calculate the distance between two matrix. And the code is generated for matrixs with all demisions. The cost function is used to get the cost of data with respect of certain centers.  The kmeans function use the traditional method to do the kmeans clustering, expect that the initial centers are from KII and kpp. 
   

In [7]:
def dis(a,b):
    return np.sum((a[None,:] - b[:, None])**2, -1)**0.5

In [8]:
%%cython -a
import numpy as np
import cython
def dis_cy(a,b):
    cdef int i, j
    cdef int m, n, l
    d=(a[None,:] - b[:, None])**2
    m,n,l=d.shape
    dis = np.zeros([m,n])
    for i in range(m):
        for j in range(n):
            dis[i,j]=(sum(d[i][j]))**0.5
    return dis

In [9]:
def cost(a,b):
    return sum(dis(a,b).min(axis=0))

In [10]:
%%cython -a
import cython
import numpy as np
from __main__ import dis_cy
def cost_cy(a,b):
#     d=np.sum((a[None,:] - b[:, None])**2, -1)**0.5
    d = dis_cy(a, b)
    return  sum(d.min(axis=0))

In [20]:
def kmeans(data, ncenter, center, maxiter=1000, maxtol=10e-5):
    centroids=[center]
    i = 0
    diff=100
    while (diff>maxtol and i<maxiter):
        # assign data points to clusters
        
        index=np.argmin(dis(data,centroids[i]),axis=0)
        

        clusters=[data[index==i] for i in range(ncenter)]

        # recalculate centroids
        centroids.append(np.concatenate([np.mean(cluster,axis=0) for cluster in clusters]).reshape(ncenter,-1))
        diff=sum(np.sum((centroids[i+1]-centroids[i])**2,-1)**0.5)

        i+= 1
    return centroids, clusters,i

In [21]:
%%cython -a
import cython
import numpy as np
from __main__ import dis_cy, dis


def kmeans_cy(data, ncenter, center, maxiter=1000, maxtol=10e-5):
    #m,center.shape
    centroids=[center]
    centroidsa=np.array(centroids)[0]
    i = 0
    diff=100
    while (diff>maxtol and i<maxiter):
        index=np.argmin(dis(data,centroids[i]),axis=0)
        clusters=[data[index==j] for j in range(ncenter)]
      
        temp = np.array([np.mean(cluster,axis=0) for cluster in clusters]).reshape(ncenter,-1)
       
        centroidsa=np.concatenate((centroidsa,temp),axis=1)
        
        centroids.append(temp)
        
     
        diff=sum(np.sum((centroids[i+1]-centroids[i])**2,-1)**0.5)
       
       
        i+= 1
    return centroids, clusters,i

In [11]:
def kII(x,k,l):
    index=set()
    index.add(np.random.choice(x.shape[0],1)[0])
    cost=sum(dis(x,x[list(index),]).min(axis=0))
    i=1
    while (i<np.log(cost)):
        prob=dis(x,x[list(index),]).min(axis=0)/sum(dis(x,x[list(index),]).min(axis=0)) 
        cp=np.random.choice(x.shape[0],l,p=prob,replace=True)
        index=index.union(set(cp))
        i=i+1
    center0=x[list(index),]
    w=np.sum(np.argsort(dis(center0,center0),axis=1)==1,axis=0)
    center1=kpp(center0,k,w)
    return center1

We wrote K-mean ++ algorithm(kpp) for comparison. The function has data x, number of center k and weigh as inputs. In particular, the default weight is set as 1. Also, it produces the initial center.

In [12]:
def kpp(x,k,weight=1):
    index=[]
    index.append(np.random.choice(x.shape[0],1)[0])
    while(len(index)<k):
        
        prob=dis(x,x[index,]).min(axis=0)*weight/sum(dis(x,x[index,]).min(axis=0)*weight)  
        index.append(np.random.choice(x.shape[0],1,p=prob)[0])
       
    return x[index,]

When use the same data set to test kpp and kII. We found KII is much slower than kpp. Therefore, it is necessary to use cython as an optimization. The function is called KII_cy. As a result, cython dramatically speed up the convergence. Meanwhile,  cython verisions for kpp and kmeans are created for optimization. They are called kpp_cy and kmeans_cy


In [13]:
%%cython -a
import cython
import numpy as np
def kII_cy(x,k,l):
    index=set()
    index.add(np.random.choice(x.shape[0],1)[0])
    d=d=np.sum((x[None,:] - x[list(index),][:, None])**2, -1)**0.5
    cost=sum(d.min(axis=0))
    i=1
    fyi=int(np.log(cost))
    prob=d.min(axis=0)/sum(d.min(axis=0))
    for i in range(fyi): 
        cp=np.random.choice(x.shape[0],l,p=prob,replace=True)
        index=index.union(set(cp))
        
    center0=x[list(index),]
    dc=np.sum((center0[None,:] - center0[:, None])**2, -1)**0.5
    w=np.sum(np.argsort(dc,axis=1)==1,axis=0)
    indexpp=[]
    indexpp.append(np.random.choice(center0.shape[0],1)[0])
    while(len(indexpp)<k):
        d=np.sum((center0[None,:] - center0[indexpp,][:, None])**2, -1)**0.5
        prob=d.min(axis=0)*w/sum(d.min(axis=0)*w)  
        indexpp.append(np.random.choice(center0.shape[0],1,p=prob)[0])
        center1=center0[indexpp,]
    return center1

In [14]:
%%cython -a
import cython
import numpy as np
def kpp_cy(x,k,weight=1):
    index=[]
    index.append(np.random.choice(x.shape[0],1)[0])
    while(len(index)<k):
        d=np.sum((x[None,:] - x[index,][:, None])**2, -1)**0.5
        prob=d.min(axis=0)*weight/sum(d.min(axis=0)*weight)  
        index.append(np.random.choice(x.shape[0],1,p=prob)[0])
       
    return x[index,]

In [15]:
%%cython -a
import cython
import numpy as np
def simudata_cy(ncenter,nsize,dim):
    mean=[0]*dim
    cov=np.diag(np.random.choice([1,10,100],dim,replace=True))
    center=np.random.multivariate_normal(mean, cov, ncenter)
    s=np.random.multivariate_normal(center[0,], np.eye(dim),nsize)
    sample=np.concatenate((s, 0*np.ones(nsize)[:,None]), axis=1)
    for i in range(ncenter-1):
        s=np.random.multivariate_normal(center[i+1,], np.eye(dim),nsize)
        s=np.concatenate((s, (i+1)*np.ones(nsize)[:,None]), axis=1)
        sample=np.concatenate((sample,s),axis=0)
    return center, sample

One advantage of K-MeansII is that the iterations can be easily parallelized.  

In [16]:
def choice(k,l,prob):
    np.random.seed()
    return np.random.choice(k,l,p=prob,replace=True)
    
def ff(arg):
    return choice(*arg)

In [17]:
def kII_pal(x,k,l):
    index=set()
    index.add(np.random.choice(x.shape[0],1)[0])
    d=np.sum((x[None,:] - x[list(index),][:, None])**2, -1)**0.5
    cost=sum(d.min(axis=0))
    i=1
    fyi=int(np.log(cost))
    prob=d.min(axis=0)/sum(d.min(axis=0))


    with mp.Pool(processes=16) as pool:
        index0=pool.map(ff,[(x.shape[0],l,prob) for i in range(fyi)])
    pool.close()
    
    index=index.union(set(np.concatenate(index0,axis=0)))
    
    center0=x[list(index),]
    dc=np.sum((center0[None,:] - center0[:, None])**2, -1)**0.5
    w=np.sum(np.argsort(dc,axis=1)==1,axis=0)
    indexpp=[]
    indexpp.append(np.random.choice(center0.shape[0],1)[0])
    while(len(indexpp)<k):
        d=np.sum((center0[None,:] - center0[indexpp,][:, None])**2, -1)**0.5
        prob=d.min(axis=0)*w/sum(d.min(axis=0)*w)  
        indexpp.append(np.random.choice(center0.shape[0],1,p=prob)[0])
        center1=center0[indexpp,]
    return center1

4 Results and Camparsion
== 

4.1 Camparsion of Running Time
--

To campare the efficiency of K-Means++ and K-MeansII algorithms, we cacluated the running time based on the dataset, which were simulated from 50 centers with individual sample size of 1000. The critical for covergence is the distance between centers form current iteration and former interation, and the threshold is 10e-5. In parituclar, the distance less than 10e-5 means convergence. Five sets of starting points were drawn from the same simulated data and plugged in the k-means iterations. The results in the plot is the average running time for each sets when the assumed the number of centers equls k'. 

In [ ]:
ncenter=50
nsample=1000
dim=15
center,sample=simudata_cy(ncenter,nsample,dim)
data=sample[:,0:dim]

def timer0(ncenter):
    start = time.clock()
    center0cy=kpp_cy(data,ncenter)
    kmeans_cy(data, ncenter, center=center0cy, maxiter=1000, maxtol=10e-5)
    return time.clock() - start
def timer1(ncenter):
    l=int(0.5*ncenter)
    start = time.clock()
    center0cy=kII_cy(data,ncenter,l)
    kmeans_cy(data, ncenter, center=center0cy, maxiter=1000, maxtol=10e-5)
    return time.clock() - start

In [ ]:
t0=[]
t1=[]
for i in range(5):
    with mp.Pool(processes=16) as pool:
        t0.append(pool.map(timer0,[i for i in range(10,ncenter+20)[::5]]))
    pool.close()
    with mp.Pool(processes=16) as pool:
        t1.append(pool.map(timer1,[i for i in list(range(10,ncenter+20)[::5])]))
    pool.close()

In [30]:
plt.plot(range(10,ncenter+20)[::5],np.mean(t0,axis=0),label='K-Means++',marker='o')
plt.plot(range(10,ncenter+20)[::5],np.mean(t1,axis=0),label='K-MeansII',marker='o')
plt.ylabel('Running Time (s)')
plt.xlabel("Assumed # of Centers (y')")

NameError: name 'ncenter' is not defined

As shown i ¥n the plot, K-MeansII leads to fewer iterations and a faster convergence of the Lloyd’s iteration while the number of clusters increases. 

4.2 Camparsion of Model Cost
--

In the comparision of the quality of the coverage for K-Means++ adn K-MeansII. we use the same setting as the 4.1 section and compare the cost of the k-means algorithm at fifth iteration.

In [ ]:
def cost0(ncenter):
    center0cy=kpp_cy(data,ncenter)
    center0,cluster0, itera0 =kmeans_cy(data, ncenter, center=center0cy, maxiter=5, maxtol=10e-5)
    return modelcost_cy(center0[-1],cluster0)
def cost1(ncenter):
    l=int(0.5*ncenter)
    center0cy=kII_cy(data,ncenter,l)
    center0,cluster0, itera0 =kmeans_cy(data, ncenter, center=center0cy, maxiter=5, maxtol=10e-5)
    return modelcost(center0[-1],cluster0)

In [ ]:
c0=[]
c1=[]
for i in range(5):
    with mp.Pool(processes=16) as pool:
        c0.append(pool.map(cost0,[i for i in range(10,ncenter+20)[::5]]))
    pool.close()
    with mp.Pool(processes=16) as pool:
        c1.append(pool.map(cost1,[i for i in range(10,ncenter+20)[::5]]))
    pool.close()

In [ ]:
plt.plot(range(10,ncenter+20)[::5],np.mean(c0,axis=0),label='K-Means++',marker='o')
plt.plot(range(10,ncenter+20)[::5],np.mean(c1,axis=0),label='K-MeansII',marker='o')
plt.ylabel('Cost')
plt.xlabel("Assumed # of Centers (y')")

The cost of these two initialized methods are no substantially different.

Therefore the K-MeansII algorithm achieve better efficiency without loosing quality. Besides, it is embarrassingly parallel and hence, it admits easy realization in any parallel computational model.